<center><h1 class="text-center h3" id="loading-app">Setting things up for you... Thanks for waiting!</h1></center>

In [ ]:
import component.auth

In [ ]:
import sepal_ui.sepalwidgets as sw
from component.widget.custom_widgets import (
    CustomDrawerItem,
    CustomNavDrawer,
    CustomAppBar,
    CustomApp,
)

from component.model.app_model import AppModel
from component.tile.custom_aoi_tile import AoiTile
from component.tile.dashboard_tile import DashboardTile
from component.tile.map_tile import MapTile
from component.tile.questionnaire_tile import QuestionnaireTile
from component.tile.recipe_tile import RecipeTile

from component.message import cm
import component.frontend  # we need this line

In [ ]:
app_model = AppModel()

In [ ]:
about_tile = sw.TileAbout(cm.app.about)
about_tile.set_title("")
disclaimer_tile = sw.TileDisclaimer()

In [ ]:
aoi_tile = AoiTile()
questionnaire_tile = QuestionnaireTile()
map_tile = MapTile(app_model)
dashboard_tile = DashboardTile()
recipe_tile = RecipeTile(
    app_model, aoi_tile, questionnaire_tile, map_tile, dashboard_tile
)

In [ ]:
# Create an appBar
app_bar = CustomAppBar(cm.app.title)

In [ ]:
app_content = [
    about_tile,
    aoi_tile,
    questionnaire_tile,
    map_tile,
    dashboard_tile,
    recipe_tile,
    disclaimer_tile,
]

In [ ]:
aux_drawers = {
    "recipe_tile": {
        "title": cm.app.drawer.recipe,
        "icon": "mdi-note-text",
    },
    "about_tile": {
        "title": cm.app.drawer.about,
        "icon": "mdi-help-circle",
    },
}

app_drawers = {
    "aoi_tile": {
        "title": cm.app.drawer.aoi,
        "icon": "mdi-map-marker-check",
    },
    "questionnaire_tile": {
        "title": cm.app.drawer.question,
        "icon": "mdi-file-question",
    },
    "map_tile": {
        "title": cm.app.drawer.map,
        "icon": "mdi-map",
    },
    "dashboard_tile": {
        "title": cm.app.drawer.dashboard,
        "icon": "mdi-view-dashboard",
    },
}
aux_items = [
    CustomDrawerItem(**aux_drawers[key], card=key) for key in aux_drawers.keys()
]

app_items = [
    CustomDrawerItem(**app_drawers[key], card=key, model=app_model, bind_var="ready")
    for key in app_drawers.keys()
]

items = [aux_items[0]] + app_items + [aux_items[-1]]

In [ ]:
code_link = "https://github.com/sepal-contrib/se.plan"
wiki_link = "https://docs.sepal.io/en/latest/modules/dwn/seplan.html"
issue_link = "https://github.com/sepal-contrib/se.plan/issues/new"

app_drawer = CustomNavDrawer(
    items, code=code_link, wiki=wiki_link, issue=issue_link, app_model=app_model
)

In [ ]:
# build the Html final app by gathering everything
app = CustomApp(
    app_model, tiles=app_content, appBar=app_bar, navDrawer=app_drawer
).show_tile("about_tile")

In [ ]:
app